In [3]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import struct
import cartopy
from cartopy import crs as ccrs
import matplotlib 
from matplotlib import pyplot as plt
import os
from os.path import join, exists
from os import mkdir
import scipy
import netCDF4
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator, LongitudeLocator)
import pandas as pd
import matplotlib.path as mpath
from matplotlib.colors import TwoSlopeNorm 
from sklearn.model_selection import train_test_split
from scipy.fft import fft, ifft
from scipy import signal
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import pickle

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
import importlib
import math
import keras
import random
import importlib
import MM_util_AI
importlib.reload(MM_util_AI)
import warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")
import shap
import matplotlib.colors as colors
with open("/scratch/hz1994/blocking/data_MMmodel/filepath.txt","r") as fi:
    for ln in fi:
        if ln.startswith("Reduced_dim_variables"):
            rd_path=ln.strip().split('\t')[1]
        if ln.startswith("TMindex_filepath"):
            TMindex_path=ln.strip().split('\t')[1]   
        if ln.startswith("dimensionalized_filepath"):
            dim_path=ln.strip().split('\t')[1]   
        if ln.startswith("nondimensionalized_filepath"):
            nondim_path=ln.strip().split('\t')[1]
        if ln.startswith("conditionedT_filepath" ):
            train_path=ln.strip().split('\t')[1]
        if ln.startswith("model_filepath" ):
            models_path=ln.strip().split('\t')[1]
        if ln.startswith("fig_filepath" ):
            fig_path=ln.strip().split('\t')[1]            
            
print(rd_path)
print(TMindex_path)
print(dim_path)
print(train_path)
print(models_path)
print(fig_path)

latitudes = np.load(dim_path+'dataX_lat.npy')
longitudes = np.load(dim_path+'dataX_lon.npy')
lon_dict=dict(zip(np.arange(longitudes.size),longitudes))
lat_dict=dict(zip(np.arange(latitudes.size),latitudes))

/ext3/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/scratch/hz1994/blocking/data_MMmodel/reduceddim/
/scratch/hz1994/blocking/data_MMmodel/TMindex/
/scratch/hz1994/blocking/data_MMmodel/dim/
/scratch/hz1994/blocking/data_MMmodel/conditionT/
/scratch/hz1994/blocking/data_MMmodel/CNNmodels/
/scratch/hz1994/blocking/data_MMmodel/fig_MMmodel/


# SHAP value for a neural network at test dataset

In [ ]:
T=1
name="1250k_lowpass3dys"
setA="T"
train_path_setA=train_path+setA+'/'
models_path_setA=models_path+setA+'/'
epsilon=0.0
data_amount='1000.0k'
cnnsize='normal'
random_seed=39
subname="/batches_resampling_"+"data_"+data_amount+"_"+cnnsize+"_cnn_"+\
"regularize"+"_%.3e"%epsilon+"_rs_%d/"%random_seed
model = MM_util_AI.make_model((18,90,3))
epoch=5
weightpath=models_path_setA+'models_T_%d_committor_'%T+name+\
                      subname+"cp-%04d.ckpt"%epoch
model.load_weights(weightpath).expect_partial()

data_preprocessed=np.load(train_path_setA+"/full/"+"data_X_T%d_"%T+name+"_proc.npy")
Ysparse=np.load(train_path_setA+"/full/"+"data_Y_T%d_"%T+name+".npy")
test_idx=np.load("/scratch/hz1994/blocking/data_MMmodel/conditionT/T/test_idx.npy").squeeze()
test_data=data_preprocessed[ test_idx  ]

pred=model.predict(test_data)
ypred=pred[:,1]>pred[:,0]


path="/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/shap_values_list_epoch_%d/"%epoch
if not os.path.exists(path):
    print(path)
    os.makedirs(path)

background=test_data[::test_idx.shape[0]//100]
e = shap.DeepExplainer(model, background)
shap_values = e.shap_values(test_data)
shap_values_list=[]
for i in range(test_data.shape[0]):
    if i%1000==0:
        print(i)
    shap_values = e.shap_values(test_data[[i]])
    shap_values_list.append(shap_values)

np.save(path+"shap_values_list_epoch_%d"%epoch, shap_values_list)